# Exercise 11 - Pass Neural Net Weights Between Processes

**GOAL:** The goal of this exercise is to show how to send neural network weights between workers and the driver.

For more details on using Ray with TensorFlow, see the documentation at http://ray.readthedocs.io/en/latest/using-ray-with-tensorflow.html.

### Concepts for this Exercise - Getting and Setting Neural Net Weights

Since pickling and unpickling a TensorFlow graph can be inefficient or may not work at all, it is most efficient to ship the weights between processes as a dictionary of numpy arrays (or as a flattened numpy array).

We provide the helper class `ray.experimental.TensorFlowVariables` to help with getting and setting weights. Similar techniques should work other neural net libraries.

Consider the following neural net definition.

```python
import tensorflow as tf

x_data = tf.placeholder(tf.float32, shape=[100])
y_data = tf.placeholder(tf.float32, shape=[100])

w = tf.Variable(tf.random_uniform([1], -1.0, 1.0))
b = tf.Variable(tf.zeros([1]))
y = w * x_data + b

loss = tf.reduce_mean(tf.square(y - y_data))
optimizer = tf.train.GradientDescentOptimizer(0.5)
grads = optimizer.compute_gradients(loss)
train = optimizer.apply_gradients(grads)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
```

Then we can use the helper class as follows.

```python
variables = ray.experimental.TensorFlowVariables(loss, sess)
# Here 'weights' is a dictionary mapping variable names to the associated
# weights as a numpy array.
weights = variables.get_weights()
variables.set_weights(weights)
```

Note that there are analogous methods `variables.get_flat` and `variables.set_flat`, which concatenate the weights as a single array instead of a dictionary.

```python
# Here 'weights' is a numpy array of all of the neural net weights
# concatenated together.
weights = variables.get_flat()
variables.set_flat(weights)
```

In this exercise, we will use an actor containing a neural network and implement methods to extract and set the neural net weights.

**WARNING:** This exercise is more complex than previous exercises.

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import ray
import tensorflow as tf
import time

In [2]:
ray.init(num_cpus=4, include_webui=False, ignore_reinit_error=True)

2020-02-28 09:15:39,393	WARNING services.py:586 -- setpgrp failed, processes may not be cleaned up properly: [Errno 1] Operation not permitted.
2020-02-28 09:15:39,444	INFO resource_spec.py:212 -- Starting Ray with 9.08 GiB memory available for workers and up to 4.55 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-02-28 09:15:39,873	WARNING services.py:1403 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This may slow down performance! You may be able to free up space by deleting files in /dev/shm or terminating any running plasma_store_server processes. If you are inside a Docker container, you may need to pass an argument with the flag '--shm-size' to 'docker run'.


{'node_ip_address': '172.17.0.2',
 'redis_address': '172.17.0.2:25515',
 'object_store_address': '/tmp/ray/session_2020-02-28_09-15-39_390291_999/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-02-28_09-15-39_390291_999/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2020-02-28_09-15-39_390291_999'}

The code below defines a class containing a simple neural network.

**EXERCISE:** Implement the `set_weights` and `get_weights` methods. This should be done using the `ray.experimental.TensorFlowVariables` helper class.

In [3]:
@ray.remote
class SimpleModel(object):
    def __init__(self):
        x_data = tf.placeholder(tf.float32, shape=[100])
        y_data = tf.placeholder(tf.float32, shape=[100])

        w = tf.Variable(tf.random_uniform([1], -1.0, 1.0))
        b = tf.Variable(tf.zeros([1]))
        y = w * x_data + b

        self.loss = tf.reduce_mean(tf.square(y - y_data))
        optimizer = tf.train.GradientDescentOptimizer(0.5)
        grads = optimizer.compute_gradients(self.loss)
        self.train = optimizer.apply_gradients(grads)

        init = tf.global_variables_initializer()
        self.sess = tf.Session()

        # Here we create the TensorFlowVariables object to assist with getting
        # and setting weights.
        self.variables = ray.experimental.TensorFlowVariables(self.loss, self.sess)

        self.sess.run(init)

    def set_weights(self, weights):
        """Set the neural net weights.
        
        This method should assign the given weights to the neural net.
        
        Args:
            weights: Either a dict mapping strings (the variable names) to numpy
                arrays or a single flattened numpy array containing all of the
                concatenated weights.
        """
        # EXERCISE: You will want to use self.variables here.
        self.variables.set_weights(weights)

    def get_weights(self):
        """Get the neural net weights.
        
        This method should return the current neural net weights.
        
        Returns:
            Either a dict mapping strings (the variable names) to numpy arrays or
                a single flattened numpy array containing all of the concatenated
                weights.
        """
        # EXERCISE: You will want to use self.variables here.
        self.variables.get_weights()

Create a few actors.

In [4]:
actors = [SimpleModel.remote() for _ in range(4)]

**EXERCISE:** Get the neural net weights from all of the actors.

In [7]:
weights = ray.get([a.get_weights.remote() for a in actors])

RayTaskError(AttributeError): [36mray::SimpleModel.__init__()[39m (pid=1045, ip=172.17.0.2)
  File "python/ray/_raylet.pyx", line 437, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 449, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 450, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 452, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 430, in ray._raylet.execute_task.function_executor
  File "<ipython-input-3-1672b9cc177d>", line 4, in __init__
AttributeError: module 'tensorflow' has no attribute 'placeholder'

**EXERCISE:** Average all of the neural net weights.

**NOTE:** This will be easier to do if you chose to use `get_flat`/`set_flat` instead of `get_weights`/`set_weights` in the implementation of `SimpleModel.set_weights` and `SimpleModel.get_weights` above..

In [ ]:
raise Exception('Implement this.')

**EXERCISE:** Set the average weights on the actors.

In [ ]:
raise Exception('Implement this.')

**VERIFY:** Check that all of the actors have the same weights.

In [ ]:
weights = ray.get([actor.get_weights.remote() for actor in actors])

for i in range(len(weights)):
    np.testing.assert_equal(weights[i], weights[0])

print('Success! The test passed.')